In [1]:
from selenium import webdriver
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
import time
import lxml
from pandas import DataFrame as df
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import pyvis

In [2]:
def linkedin_scraper(company, role, scroll_time):
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
    
    #sign in to your profile
    driver.get('https://www.linkedin.com/uas/login?session_redirect=https%3A%2F%2Fwww%2Elinkedin%2Ecom%2Ffeed%2F&fromSignIn=true&trk=cold_join_sign_in')
    #find username field
    username = driver.find_element("id", "username")
    # Enter Your Email Address
    username.send_keys("arjunkmrm@gmail.com")  
    # entering password
    pword = driver.find_element("id", "password")
    # Enter Your Password
    pword.send_keys("arjlinZ420713~~")        
    #click login button
    driver.find_element("xpath", "//button[@type='submit']").click()
    
    #go to target company
    # Opening Kunal's Profile
    # paste the URL of Kunal's profile here
    profile_url = "https://www.linkedin.com/company/" + company + "/people/"
    
    driver.get(profile_url)        # this will open the link

    filter = driver.find_element("xpath", "//*[@id='people-search-keywords']")
    #id="people-search-keywords"
    filter.send_keys("sustainability")
    filter.send_keys(Keys.RETURN)

    start = time.time()
  
    # will be used in the while loop
    initialScroll = 0
    finalScroll = 1000
    
    while True:
        driver.execute_script(f"window.scrollTo({initialScroll}, {finalScroll})")
        # this command scrolls the window starting from
        # the pixel value stored in the initialScroll 
        # variable to the pixel value stored at the
        # finalScroll variable
        initialScroll = finalScroll
        finalScroll += 1000
    
        # we will stop the script for 3 seconds so that 
        # the data can load
        time.sleep(3)
        # You can change it as per your needs and internet speed
    
        end = time.time()
    
        # We will scroll for 20 seconds.
        # You can change it as per your needs and internet speed
        if round(end - start) > scroll_time:
            break
        
    src = driver.page_source
  
    # Now using beautiful soup
    soup = BeautifulSoup(src, 'lxml')

    #sustainability related roles
    sustainability_roles_html = soup.find_all(
    'div', {'class': 'artdeco-entity-lockup__subtitle ember-view'})
    sustainability_roles = []

    #names
    sustainability_names_html = soup.find_all(
    'div', {'class': ['artdeco-entity-lockup__title ember-view']})
    sustainability_names = []
    
    for role in sustainability_roles_html:
        sustainability_roles.append(role.text.strip())

    for name in sustainability_names_html:
        sustainability_names.append(name.text.strip())

    del sustainability_names[0]
    
    sust_df = df({'name':sustainability_names, 'role':sustainability_roles})
    #names_df = df({'name':sustainability_names})
    sust_df['company'] = company

    return(sust_df)

In [7]:
msft = linkedin_scraper('microsoft', 'sustainability', 180)
meta = linkedin_scraper('meta', 'sustainability', 180)
goog = linkedin_scraper('google', 'sustainability', 180)
appl = linkedin_scraper('apple', 'sustainability', 180)
amzn = linkedin_scraper('amazon', 'sustainability', 180)

troles_df = df()
troles_df = pd.concat([msft, meta, goog, appl, amzn])
troles_df



,name,role,company
0,Kavickumar Muruganathan,Sustainability | ESG | Technology,microsoft
1,Benson Ang,Digital Transformation | Operational Excellenc...,microsoft
2,Leo Wirawan,Energy and Sustainability,microsoft
3,Srikant Kadambi,"Managing Director, Energy & Sustainability, As...",microsoft
4,Jim Bullock,Energy and Sustainability at Microsoft,microsoft
...,...,...,...
559,Céline Hasboun,Marketing Manager and Sustainability Ambassado...,amazon
560,Sukhmeet Singh,"Manager II, Amazon Robotics | DEI | Corporate ...",amazon
561,"Bella Nguyen, PhD","Sustainable Buildings, Amazon Transportation S...",amazon
562,Archit Koti,BEV | FCEV | Sustainability | ZEV Strategy,amazon


In [8]:
#compression_opts = dict(method='zip', archive_name='roles.csv')
troles_df.to_csv('tech_roles.csv', index=False)

In [9]:
dbs = linkedin_scraper('dbs-bank', 'sustainability', 180)
dbs.head()

ValueError: All arrays must be of the same length

In [9]:
scb = linkedin_scraper('standardchartered', 'sustainability', 180)
bnpp = linkedin_scraper('bnp-paribas', 'sustainability', 180)
citi = linkedin_scraper('citi', 'sustainability', 180)
hsbc = linkedin_scraper('hsbc', 'sustainability', 180)
jpm = linkedin_scraper('jpmorgan', 'sustainability', 180)

broles_df = df()
broles_df = pd.concat([scb, bnpp, citi, hsbc, jpm])
broles_df


[WDM] - Downloading: 100%|██████████| 7.46M/7.46M [00:00<00:00, 17.9MB/s]


,name,role,company
0,Zhi Ling Wong,"Head, Reputational & Sustainability Risk Frame...",standardchartered
1,Rahul Prem,Program Manager | Financial Markets | Sustaina...,standardchartered
2,"Alyssa Lin, CESGA",Make the world a better place.,standardchartered
3,Glenn Ho 何伟良,Sustainable Finance I Climate Risk Management ...,standardchartered
4,Faisal Abdul Karim,Innovation leader in Banking | FinTech | Risk ...,standardchartered
...,...,...,...
137,Jane Bai,Mechanical Engineering and Economics Student a...,jpmorgan
138,LinkedIn Member,Lead Tableau and Alteryx developer with extens...,jpmorgan
139,Marcos Abud,Sr. Implementation Manager at J.P. Morgan,jpmorgan
140,Kate Liu,Software Engineer,jpmorgan


In [10]:
#compression_opts = dict(method='zip', archive_name='roles.csv')
broles_df.to_csv('bank_roles.csv', index=False)

In [8]:
dbs.to_csv('dbs_roles.csv', index=False)